<a href="https://colab.research.google.com/github/pko89403/Recsys_test/blob/master/Wide%26Deep_Data_Model_subscriton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wide - And - Deep - Pytorch
1. Wide : 스파스한 피처들, 원-핫 인코딩 되어 아웃풋 뉴런과 바로 연결됨
2. Deep Dense : 카테고리 피처들, 임베딩으로 표횐됨, 연속성 피처들이 덴스 레이어로 보내짐
3. Deep Text : 워드 임베딩들 보내지는 RNN들의 스택으로
4. Deep Image : RGB 이미지들 CNN으로 보내지는, ResNet

# 데이터 전처리
https://github.com/Srifox/Wide-and-Deep-PyTorch/blob/master/demo1_prepare_data.ipynb

In [337]:
import pandas as pd
import numpy as np

raw_data_path = "/content/drive/My Drive/data/adult_data/adult-data.csv"
raw_data = pd.read_csv(raw_data_path)
raw_data.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital-gain,capital-loss,wrk_hrs_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


age 컬럼의 카테고리 버전을 만든다.

In [0]:
raw_data['age_buckets'] = pd.cut(raw_data.age, bins=[16, 25, 30, 35, 40, 45, 50, 55, 60, 91], labels=np.arange(9))

Logist Regression에 사용하기 위한 라벨 만들기

In [339]:
raw_data['income_label'] = (raw_data['income'].apply(lambda x: ">50K" in x)).astype(int)
raw_data.drop('income', axis=1, inplace=True)
raw_data.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital-gain,capital-loss,wrk_hrs_per_week,native_country,age_buckets,income_label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,3,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,5,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,3,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,6,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,1,0


wide-side 모델에서 사용할 컬럼을 선택하기

In [0]:
wide_cols = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'age_buckets']
crossed_cols = (['education', 'occupation'], ['native_country', 'occupation'])

기존 논문의 crossed 에 대한 표현이 좀 더 합리적으로 변경 되었다.   
전에는 crossed columns 에 대한 표현이 (true, true) -> 1, (다른 페어) -> 0
- education_occupation
- native_country_occupation

In [341]:
df_adult_wide = raw_data.copy()[wide_cols]
crossed_columns = []
for cols in crossed_cols:
  colname = '_'.join(cols)
  df_adult_wide[colname] = df_adult_wide[cols].apply(lambda x: '-'.join(x), axis=1)
  crossed_columns.append(colname)
df_adult_wide['age_buckets'].astype(str)
df_adult_wide.head() 



,workclass,education,marital_status,occupation,relationship,race,sex,native_country,age_buckets,education_occupation,native_country_occupation
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,3,Bachelors- Adm-clerical,United-States- Adm-clerical
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,5,Bachelors- Exec-managerial,United-States- Exec-managerial
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,3,HS-grad- Handlers-cleaners,United-States- Handlers-cleaners
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,6,11th- Handlers-cleaners,United-States- Handlers-cleaners
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,1,Bachelors- Prof-specialty,Cuba- Prof-specialty


pd.get_dummies()를 이용해 가변수 (dummy var) 만들기 - 원-핫 인코딩

In [342]:
dummy_cols = wide_cols + crossed_columns
df_adult_wide = pd.get_dummies(df_adult_wide, columns=dummy_cols)
print(df_adult_wide.shape)

(32561, 770)


In [343]:
df_adult_wide.head()

,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,education_ 10th,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,education_ 7th-8th,education_ 9th,education_ Assoc-acdm,education_ Assoc-voc,education_ Bachelors,education_ Doctorate,education_ HS-grad,education_ Masters,education_ Preschool,education_ Prof-school,education_ Some-college,marital_status_ Divorced,marital_status_ Married-AF-spouse,marital_status_ Married-civ-spouse,marital_status_ Married-spouse-absent,marital_status_ Never-married,marital_status_ Separated,marital_status_ Widowed,occupation_ ?,occupation_ Adm-clerical,occupation_ Armed-Forces,occupation_ Craft-repair,occupation_ Exec-managerial,occupation_ Farming-fishing,occupation_ Handlers-cleaners,occupation_ Machine-op-inspct,...,native_country_occupation_ Trinadad&Tobago- Machine-op-inspct,native_country_occupation_ Trinadad&Tobago- Other-service,native_country_occupation_ Trinadad&Tobago- Prof-specialty,native_country_occupation_ Trinadad&Tobago- Sales,native_country_occupation_ Trinadad&Tobago- Tech-support,native_country_occupation_ Trinadad&Tobago- Transport-moving,native_country_occupation_ United-States- ?,native_country_occupation_ United-States- Adm-clerical,native_country_occupation_ United-States- Armed-Forces,native_country_occupation_ United-States- Craft-repair,native_country_occupation_ United-States- Exec-managerial,native_country_occupation_ United-States- Farming-fishing,native_country_occupation_ United-States- Handlers-cleaners,native_country_occupation_ United-States- Machine-op-inspct,native_country_occupation_ United-States- Other-service,native_country_occupation_ United-States- Priv-house-serv,native_country_occupation_ United-States- Prof-specialty,native_country_occupation_ United-States- Protective-serv,native_country_occupation_ United-States- Sales,native_country_occupation_ United-States- Tech-support,native_country_occupation_ United-States- Transport-moving,native_country_occupation_ Vietnam- ?,native_country_occupation_ Vietnam- Adm-clerical,native_country_occupation_ Vietnam- Craft-repair,native_country_occupation_ Vietnam- Exec-managerial,native_country_occupation_ Vietnam- Farming-fishing,native_country_occupation_ Vietnam- Handlers-cleaners,native_country_occupation_ Vietnam- Machine-op-inspct,native_country_occupation_ Vietnam- Other-service,native_country_occupation_ Vietnam- Prof-specialty,native_country_occupation_ Vietnam- Sales,native_country_occupation_ Vietnam- Tech-support,native_country_occupation_ Vietnam- Transport-moving,native_country_occupation_ Yugoslavia- Adm-clerical,native_country_occupation_ Yugoslavia- Craft-repair,native_country_occupation_ Yugoslavia- Exec-managerial,native_country_occupation_ Yugoslavia- Farming-fishing,native_country_occupation_ Yugoslavia- Machine-op-inspct,native_country_occupation_ Yugoslavia- Other-service,native_country_occupation_ Yugoslavia- Transport-moving
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


deep-dense 사이드 모델에 사용할 컬럼을 정의한다.   
임베딩으로 표현되어 Dense 레이어로 들어갈 예정이다.

In [0]:
# embeddings_cols = [(column_name, dimension of the corresponding embeddings)]
embeddings_cols = [('workclass', 10),
                   ('education', 10),
                   ('occupation', 10),
                   ('relationship', 8),
                   ('native_country', 10)]
continuous_cols = ['age', 'wrk_hrs_per_week', 'capital-gain', 'capital-loss']

In [345]:
def_dum = 8
if type(embeddings_cols[0]) is tuple:
  emb_dim = dict(embeddings_cols)
  embeddings_cols = [emb[0] for emb in embeddings_cols]
else:
  emb_dim = {e:def_dim for e in embeddings_cols}
  embeddings_cols = embeddings_cols
deep_cols = embeddings_cols + continuous_cols
deep_cols

['workclass',
 'education',
 'occupation',
 'relationship',
 'native_country',
 'age',
 'wrk_hrs_per_week',
 'capital-gain',
 'capital-loss']

In [346]:
df_adult_deep = raw_data.copy()[deep_cols]
categorical_columns = list(df_adult_deep.select_dtypes(include=['object']).columns)
categorical_columns

['workclass', 'education', 'occupation', 'relationship', 'native_country']

카테고리컬 컬럼들을 'label encode' 한다.

In [0]:
def label_encode(df_input, cols=None, val_to_idx=None):
  df = df_input.copy()
  if cols == None:
    cols = list(df.select_dtypes(include=['object']).columns)

  if not val_to_idx:
    val_types = dict()
    for c in cols:
      val_types[c] = df[c].unique() # 각 카테고리 컬럼의 항목을 넣는다. 

      val_to_idx = dict()
      for k, v in val_types.items(): # 카테고리 컬럼 이름, 카테고리 항목
        val_to_idx[k] = {o: i for i, o in enumerate(val_types[k])}

  for k, v in val_to_idx.items():
    df[k] = df[k].apply(lambda x: v[x])
  
  return df, val_to_idx

In [0]:
df_adult_deep, encoding_dict = label_encode(df_adult_deep, cols=categorical_columns)
embeddings_encoding_dict = {k:encoding_dict[k] for k in encoding_dict if k in deep_cols}
embeddings_input = []
for k, v in embeddings_encoding_dict.items():
  embeddings_input.append((k, len(v), emb_dim[k]))

deep_column_idx = {k:v for v,k in enumerate(df_adult_deep.columns)}

In [349]:
deep_column_idx

{'age': 5,
 'capital-gain': 7,
 'capital-loss': 8,
 'education': 1,
 'native_country': 4,
 'occupation': 2,
 'relationship': 3,
 'workclass': 0,
 'wrk_hrs_per_week': 6}

In [350]:
embeddings_encoding_dict

{'education': {' 10th': 12,
  ' 11th': 2,
  ' 12th': 15,
  ' 1st-4th': 13,
  ' 5th-6th': 11,
  ' 7th-8th': 8,
  ' 9th': 4,
  ' Assoc-acdm': 6,
  ' Assoc-voc': 7,
  ' Bachelors': 0,
  ' Doctorate': 9,
  ' HS-grad': 1,
  ' Masters': 3,
  ' Preschool': 14,
  ' Prof-school': 10,
  ' Some-college': 5},
 'native_country': {' ?': 4,
  ' Cambodia': 17,
  ' Canada': 10,
  ' China': 28,
  ' Columbia': 16,
  ' Cuba': 1,
  ' Dominican-Republic': 24,
  ' Ecuador': 19,
  ' El-Salvador': 25,
  ' England': 9,
  ' France': 26,
  ' Germany': 11,
  ' Greece': 35,
  ' Guatemala': 27,
  ' Haiti': 22,
  ' Holand-Netherlands': 41,
  ' Honduras': 8,
  ' Hong': 38,
  ' Hungary': 40,
  ' India': 3,
  ' Iran': 12,
  ' Ireland': 39,
  ' Italy': 14,
  ' Jamaica': 2,
  ' Japan': 29,
  ' Laos': 20,
  ' Mexico': 5,
  ' Nicaragua': 36,
  ' Outlying-US(Guam-USVI-etc)': 32,
  ' Peru': 31,
  ' Philippines': 13,
  ' Poland': 15,
  ' Portugal': 23,
  ' Puerto-Rico': 7,
  ' Scotland': 33,
  ' South': 6,
  ' Taiwan': 21,
  '

In [351]:
embeddings_input

[('workclass', 9, 10),
 ('education', 16, 10),
 ('occupation', 15, 10),
 ('relationship', 6, 8),
 ('native_country', 42, 10)]

In [352]:
df_adult_deep.head()

,workclass,education,occupation,relationship,native_country,age,wrk_hrs_per_week,capital-gain,capital-loss
0,0,0,0,0,0,39,40,2174,0
1,1,0,1,1,0,50,13,0,0
2,2,1,2,0,0,38,40,0,0
3,2,2,2,1,0,53,40,0,0
4,2,0,3,2,1,28,40,0,0


In [353]:
from sklearn.preprocessing import StandardScaler

standardize_cols = continuous_cols
scaler = StandardScaler()

for cc in standardize_cols:
  df_adult_deep[cc] = scaler.fit_transform(raw_data[cc].values.reshape(-1, 1).astype(float))

df_adult_deep.head()

,workclass,education,occupation,relationship,native_country,age,wrk_hrs_per_week,capital-gain,capital-loss
0,0,0,0,0,0,0.030671,-0.035429,0.148453,-0.21666
1,1,0,1,1,0,0.837109,-2.222153,-0.145920,-0.21666
2,2,1,2,0,0,-0.042642,-0.035429,-0.145920,-0.21666
3,2,2,2,1,0,1.057047,-0.035429,-0.145920,-0.21666
4,2,0,3,2,1,-0.775768,-0.035429,-0.145920,-0.21666


In [0]:
from sklearn.model_selection import train_test_split

x_wide = df_adult_wide.values
y = np.array(raw_data['income_label'])

x_deep = df_adult_deep.values
cat_embed_input = embeddings_input
cat_embed_encoding_dict = embeddings_encoding_dict
deep_column_idx = deep_column_idx
continuous_cols = continuous_cols

# train / valid / test split
x_train_wide, x_valid_wide = train_test_split(x_wide, test_size=0.4, random_state=1004)
x_train_deep, x_valid_deep = train_test_split(x_deep, test_size=0.4, random_state=1004)
y_train, y_valid = train_test_split(y, test_size=0.4, random_state=1004)

x_valid_wide, x_test_wide = train_test_split(x_valid_wide, test_size=0.4, random_state=1004)
x_valid_deep, x_test_deep = train_test_split(x_valid_deep, test_size=0.4, random_state=1004)
y_valid, y_test = train_test_split(y_valid, test_size=0.4, random_state=1004)

In [0]:
from collections import namedtuple

wd_dataset = dict()
train_dataset = namedtuple('train_dataset', 'wide, deep, labels')
test_dataset = namedtuple('valid_dataset', 'wide, deep, labels')
test_dataset = namedtuple('test_dataset', 'wide, deep, labels')
wd_dataset['train_dataset'] = train_dataset(x_train_wide, x_train_deep, y_train)
wd_dataset['valid_dataset'] = train_dataset(x_valid_wide, x_valid_deep, y_valid)
wd_dataset['test_dataset'] = test_dataset(x_test_wide, x_test_deep, y_test)
wd_dataset['embeddings_input'] = embeddings_input
wd_dataset['deep_column_idx'] = deep_column_idx
wd_dataset['continuous_cols'] = continuous_cols
wd_dataset['encoding_dict'] = encoding_dict

## 모델을 만들어 보자 https://github.com/Srifox/Wide-and-Deep-PyTorch/blob/master/demo2_building_blocks.ipynb   
### Wide

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Wide(nn.Module):
  def __init__(self, wide_dim, output_dim):
    super(Wide, self).__init__()
    self.linear = nn.Linear(wide_dim, output_dim)
  def forward(self, input):
    out = self.linear(input)
    return F.sigmoid(out)

In [357]:
x_train_wide.shape

(19536, 770)

In [358]:
wide_model = Wide(wide_dim=wd_dataset['train_dataset'].wide.shape[1], output_dim=1)

wide_x_train = torch.Tensor(x_train_wide[0:4])
print(wide_x_train.shape)

result = wide_model(wide_x_train)
print(result)

torch.Size([4, 770])
tensor([[0.4731],
        [0.4958],
        [0.5336],
        [0.4976]], grad_fn=<SigmoidBackward>)


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [359]:
train_dataset = np.hstack([wd_dataset['train_dataset'].labels.reshape(-1,1), wd_dataset['train_dataset'].wide]) # 이럴려고 네임튜플을 만들었구나
train_dataset

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [0]:
optimizer = torch.optim.Adam(wide_model.parameters())
batch_size = 64
n_epochs = 10
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

In [361]:
for epoch in range(n_epochs):
  total = 0
  correct = 0

  for i, batch in enumerate(train_loader):
    x_w = torch.autograd.Variable(batch[:, 1:]).float()
    y = torch.autograd.Variable(batch[:, 0]).float()
    y = y.reshape(-1,1)
    optimizer.zero_grad() # zeroes the gradient buffers of all parameters
    y_pred = wide_model(x_w)

    loss = F.binary_cross_entropy(y_pred, y)
    loss.backward()
    optimizer.step()

    total += y.size(0)
    y_pred_cat = (y_pred > 0.5).squeeze(1).float()
  
    correct += float((y_pred_cat == y).sum().item())

  print('Epoch {} of {}, Loss: {}, Accuracy: {}'.format(epoch+1, n_epochs, round(loss.item(),3), round(correct/total,4)))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 1 of 10, Loss: 0.468, Accuracy: 47.8645
Epoch 2 of 10, Loss: 0.201, Accuracy: 45.2072
Epoch 3 of 10, Loss: 0.394, Accuracy: 43.9482
Epoch 4 of 10, Loss: 0.554, Accuracy: 43.4011
Epoch 5 of 10, Loss: 0.518, Accuracy: 43.1529
Epoch 6 of 10, Loss: 0.358, Accuracy: 43.1069
Epoch 7 of 10, Loss: 0.366, Accuracy: 42.8866
Epoch 8 of 10, Loss: 0.382, Accuracy: 42.8473
Epoch 9 of 10, Loss: 0.333, Accuracy: 42.8183
Epoch 10 of 10, Loss: 0.159, Accuracy: 42.6628


### Deep

In [362]:
print(wd_dataset['embeddings_input'])
print(wd_dataset['deep_column_idx'])

[('workclass', 9, 10), ('education', 16, 10), ('occupation', 15, 10), ('relationship', 6, 8), ('native_country', 42, 10)]
{'workclass': 0, 'education': 1, 'occupation': 2, 'relationship': 3, 'native_country': 4, 'age': 5, 'wrk_hrs_per_week': 6, 'capital-gain': 7, 'capital-loss': 8}


In [363]:
col_name, unique_vals, n_emb = wd_dataset['embeddings_input'][0]
emb_layer = nn.Embedding(unique_vals, n_emb)
print(emb_layer)

Embedding(9, 10)


In [0]:
def dense_layer(input, output):
  return nn.Sequential(
      nn.Linear(input, output),
      nn.LeakyReLU(inplace=True),
      nn.Dropout()
      )

class DeepDense(nn.Module):
  def __init__(self, embeddings_input, embeddings_encoding_dict, continuous_cols, deep_column_idx, hidden_layers, output_dim):
    super(DeepDense, self).__init__()

    self.embeddings_input = embeddings_input
    self.embeddings_encoding_dict = embeddings_encoding_dict
    self.continuous_cols = continuous_cols
    self.deep_column_idx = deep_column_idx

    for col, val, dim in embeddings_input:
      print(col, val, dim)
      setattr(self, 'emb_layer_' + col, nn.Embedding(val, dim))
    
    input_emb_dim = np.sum([emb[2] for emb in embeddings_input]) + len(continuous_cols)
    hidden_layers = [input_emb_dim] + hidden_layers
    
    self.dense = nn.Sequential()
    for i in range(1, len(hidden_layers)):
      self.dense.add_module(
          'dense_layer_{}'.format(i-1),
          dense_layer( hidden_layers[i-1], hidden_layers[i])

      )
    
    self.dense.add_module('last_linear', nn.Linear(hidden_layers[-1], output_dim))

  def forward(self, X:torch.Tensor)->torch.Tensor:
    emb = [getattr(self, 'emb_layer_' + col)(X[:, self.deep_column_idx[col]].long())
            for col, _, _ in self.embeddings_input]     
    if self.continuous_cols:
      cont_idx = [self.deep_column_idx[col] for col in self.continuous_cols]
      cont = [X[:, cont_idx].float()]
      input = torch.cat(emb+cont, 1)
    else:
      input = torch.cat(emb, 1)
    out = self.dense(input)                                                                                        
    return out

In [365]:
cat_embed_input # 유니크 항목의 수, 임베딩 차원 수

[('workclass', 9, 10),
 ('education', 16, 10),
 ('occupation', 15, 10),
 ('relationship', 6, 8),
 ('native_country', 42, 10)]

In [366]:
cat_embed_encoding_dict 

{'education': {' 10th': 12,
  ' 11th': 2,
  ' 12th': 15,
  ' 1st-4th': 13,
  ' 5th-6th': 11,
  ' 7th-8th': 8,
  ' 9th': 4,
  ' Assoc-acdm': 6,
  ' Assoc-voc': 7,
  ' Bachelors': 0,
  ' Doctorate': 9,
  ' HS-grad': 1,
  ' Masters': 3,
  ' Preschool': 14,
  ' Prof-school': 10,
  ' Some-college': 5},
 'native_country': {' ?': 4,
  ' Cambodia': 17,
  ' Canada': 10,
  ' China': 28,
  ' Columbia': 16,
  ' Cuba': 1,
  ' Dominican-Republic': 24,
  ' Ecuador': 19,
  ' El-Salvador': 25,
  ' England': 9,
  ' France': 26,
  ' Germany': 11,
  ' Greece': 35,
  ' Guatemala': 27,
  ' Haiti': 22,
  ' Holand-Netherlands': 41,
  ' Honduras': 8,
  ' Hong': 38,
  ' Hungary': 40,
  ' India': 3,
  ' Iran': 12,
  ' Ireland': 39,
  ' Italy': 14,
  ' Jamaica': 2,
  ' Japan': 29,
  ' Laos': 20,
  ' Mexico': 5,
  ' Nicaragua': 36,
  ' Outlying-US(Guam-USVI-etc)': 32,
  ' Peru': 31,
  ' Philippines': 13,
  ' Poland': 15,
  ' Portugal': 23,
  ' Puerto-Rico': 7,
  ' Scotland': 33,
  ' South': 6,
  ' Taiwan': 21,
  '

In [367]:
continuous_cols

['age', 'wrk_hrs_per_week', 'capital-gain', 'capital-loss']

In [368]:
wd_dataset['train_dataset']
wd_dataset['valid_dataset']
wd_dataset['test_dataset']

wd_dataset['embeddings_input']
wd_dataset['encoding_dict']
wd_dataset['continuous_cols']
wd_dataset['deep_column_idx']

{'age': 5,
 'capital-gain': 7,
 'capital-loss': 8,
 'education': 1,
 'native_country': 4,
 'occupation': 2,
 'relationship': 3,
 'workclass': 0,
 'wrk_hrs_per_week': 6}

In [369]:
deep = DeepDense( embeddings_input=wd_dataset['embeddings_input'],
                  embeddings_encoding_dict=wd_dataset['encoding_dict'], 
                  continuous_cols=wd_dataset['continuous_cols'],
                  deep_column_idx=wd_dataset['deep_column_idx'], 
                  hidden_layers=[32,16], 
                  output_dim=1)

workclass 9 10
education 16 10
occupation 15 10
relationship 6 8
native_country 42 10


In [370]:
deep

DeepDense(
  (emb_layer_workclass): Embedding(9, 10)
  (emb_layer_education): Embedding(16, 10)
  (emb_layer_occupation): Embedding(15, 10)
  (emb_layer_relationship): Embedding(6, 8)
  (emb_layer_native_country): Embedding(42, 10)
  (dense): Sequential(
    (dense_layer_0): Sequential(
      (0): Linear(in_features=52, out_features=32, bias=True)
      (1): LeakyReLU(negative_slope=0.01, inplace=True)
      (2): Dropout(p=0.5, inplace=False)
    )
    (dense_layer_1): Sequential(
      (0): Linear(in_features=32, out_features=16, bias=True)
      (1): LeakyReLU(negative_slope=0.01, inplace=True)
      (2): Dropout(p=0.5, inplace=False)
    )
    (last_linear): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [371]:
deep_batch = torch.from_numpy(x_train_deep[:4])
deep_batch.shape

torch.Size([4, 9])

In [372]:
out = deep(deep_batch)
out

tensor([[-0.4796],
        [-0.3287],
        [-0.2823],
        [-0.4917]], grad_fn=<AddmmBackward>)

Wide and Deep 

In [0]:
def set_method(method):
  if method == 'regression':
    return None, F.mse_loss
  if method == 'logistic':
    return torch.sigmoid, F.binary_cross_entropy
  if method == 'multiclass':
    return F.softmax, F.cross_entropy

In [0]:
def set_optimizer(model_params, opt_params):
  try:
    opt, lr, m = opt_params
  except:
    opt, lr = opt_params

  if opt == "Adam":
    return torch.optim.Adam(model_params, lr=lr)
  if opt == "Adagrad":
    return torch.optim.Adam(model_params, lr=lr)
  if opt == "RMSprop":
    return torch.optim.RMSprop(model_params, lr=lr, momentum=m)
  if opt == "SGD":
    return torch.optim.SGD(model_params, lr=lr, momentum=m)

In [0]:
class WideDeep(nn.Module):
  def __init__(self, output_dim, **params):
    super(WideDeep, self).__init__()

    self.datasets = {}
    self.output_dim = output_dim
    self.n_datasets = 1
    for k, v in params['wide'].items():
      setattr(self, k, v)
    self.wide = Wide(
      self.wide_dim,
      self.output_dim)
    
    self.datasets['deep_dense'] = self.n_datasets
    self.n_datasets += 1
    for k, v in params['deep_dense'].items():
      setattr(self, k, v)
    self.deep_dense = DeepDense(
      embeddings_input=self.embeddings_input,
      embeddings_encoding_dict=self.embeddings_encoding_dict, 
      continuous_cols=self.continuous_cols,
      deep_column_idx=self.deep_column_idx, 
      hidden_layers=self.hidden_layers, 
      output_dim=self.output_dim)
    
  def compile(self, method, optimizer):
    self.method = method,
    self.activation, self.criterion = set_method(method)
    self.optimizer = set_optimizer(self.parameters(), optimizer)

  def forward(self, input):
    wide_inp = input[0]
    wide_deep = self.wide(wide_inp)

    deep_dense_idx = self.datasets['deep_dense']
    deep_dense_out = self.deep_dense(input[deep_dense_idx])
    wide_deep.add_(deep_dense_out)

    if not self.activation:
      return wide_deep
    else:
      if( self.activation == F.softmax):
        out = self.activation(wide_deep, dim=1)
      else:
        out = self.activation(wide_deep)
      return out

In [376]:
params = dict()
params['wide'] = dict(
  wide_dim = 770
)
params['deep_dense'] = dict(
  embeddings_input=wd_dataset['embeddings_input'],
  embeddings_encoding_dict=wd_dataset['encoding_dict'], 
  continuous_cols=wd_dataset['continuous_cols'],
  deep_column_idx=wd_dataset['deep_column_idx'], 
  hidden_layers=[32,16]
)

params

{'deep_dense': {'continuous_cols': ['age',
   'wrk_hrs_per_week',
   'capital-gain',
   'capital-loss'],
  'deep_column_idx': {'age': 5,
   'capital-gain': 7,
   'capital-loss': 8,
   'education': 1,
   'native_country': 4,
   'occupation': 2,
   'relationship': 3,
   'workclass': 0,
   'wrk_hrs_per_week': 6},
  'embeddings_encoding_dict': {'education': {' 10th': 12,
    ' 11th': 2,
    ' 12th': 15,
    ' 1st-4th': 13,
    ' 5th-6th': 11,
    ' 7th-8th': 8,
    ' 9th': 4,
    ' Assoc-acdm': 6,
    ' Assoc-voc': 7,
    ' Bachelors': 0,
    ' Doctorate': 9,
    ' HS-grad': 1,
    ' Masters': 3,
    ' Preschool': 14,
    ' Prof-school': 10,
    ' Some-college': 5},
   'native_country': {' ?': 4,
    ' Cambodia': 17,
    ' Canada': 10,
    ' China': 28,
    ' Columbia': 16,
    ' Cuba': 1,
    ' Dominican-Republic': 24,
    ' Ecuador': 19,
    ' El-Salvador': 25,
    ' England': 9,
    ' France': 26,
    ' Germany': 11,
    ' Greece': 35,
    ' Guatemala': 27,
    ' Haiti': 22,
    ' Holan

In [377]:
wide_deep = WideDeep(output_dim=1, **params)

workclass 9 10
education 16 10
occupation 15 10
relationship 6 8
native_country 42 10


In [378]:
wide_deep

WideDeep(
  (wide): Wide(
    (linear): Linear(in_features=770, out_features=1, bias=True)
  )
  (deep_dense): DeepDense(
    (emb_layer_workclass): Embedding(9, 10)
    (emb_layer_education): Embedding(16, 10)
    (emb_layer_occupation): Embedding(15, 10)
    (emb_layer_relationship): Embedding(6, 8)
    (emb_layer_native_country): Embedding(42, 10)
    (dense): Sequential(
      (dense_layer_0): Sequential(
        (0): Linear(in_features=52, out_features=32, bias=True)
        (1): LeakyReLU(negative_slope=0.01, inplace=True)
        (2): Dropout(p=0.5, inplace=False)
      )
      (dense_layer_1): Sequential(
        (0): Linear(in_features=32, out_features=16, bias=True)
        (1): LeakyReLU(negative_slope=0.01, inplace=True)
        (2): Dropout(p=0.5, inplace=False)
      )
      (last_linear): Linear(in_features=16, out_features=1, bias=True)
    )
  )
)

In [0]:
wide_batch = torch.Tensor(x_train_wide[0:4])
deep_batch = torch.from_numpy(x_train_deep[:4])
wide_deep_batch = (wide_batch, deep_batch)

In [380]:
wide_deep.compile(method='regression', optimizer=["Adam", 0.01])
wide_deep(wide_deep_batch)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[0.5616],
        [0.3905],
        [0.3615],
        [0.2980]], grad_fn=<AddBackward0>)

https://github.com/Srifox/Wide-and-Deep-PyTorch/blob/master/demo3_using_it.ipynb